# Inspect prediction results from a Random Forest classifier

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import pickle
import glob
import matplotlib.pyplot as plt
from plotnine import *
from sklearn.metrics import classification_report, confusion_matrix
import read_settings

Set input and output directories

In [ ]:
# Read settings
global_settings = read_settings.check_global()
instrument = global_settings['input_data']['instrument']
rf_settings = read_settings.check_rf()
use_weights = rf_settings['use_weights']

# Directory for input data
data_dir = os.path.join('data', instrument)

# Directory for training outputs
output_dirs = glob.glob(os.path.join('output', '_'.join(['rf', 'w' if use_weights else 'nw', instrument, '*'])))
output_dirs

Choose output directory to inspect

In [ ]:
output_dir = output_dirs[0]

Glimpse at settings

In [ ]:
with open(os.path.join(output_dir, 'settings.pickle'),'rb') as settings_file:
    settings = pickle.load(settings_file)
settings

## Input data

In [ ]:
df_comp = pd.read_csv(os.path.join(output_dir, 'df_comp.csv')).set_index('classif_id')

In [ ]:
plt.figure()
df_comp.plot.bar(stacked=True, figsize=(16,8), fontsize = 14)
plt.xlabel("Classes", fontsize = 14)
plt.ylabel("Image number", fontsize = 14)
plt.legend(loc="best")
plt.title("Dataset composition for RF", fontsize = 16)
plt.show()

## Training

Read training file

In [ ]:
with open(os.path.join(output_dir, 'train_results.pickle'),'rb') as results_file:
    train_results = pickle.load(results_file)
    train_results = pd.DataFrame(train_results)

Plot gridsearch results

In [ ]:
(ggplot(train_results) +
  geom_point(aes(x='max_features', y='valid_accuracy', colour='factor(n_estimators)'))+
  facet_wrap('~min_samples_leaf', labeller = 'label_both') +
  labs(colour='n_estimators', title = 'Gridsearch results'))

## Testing

Read test file

In [ ]:
with open(os.path.join(output_dir, 'test_results.pickle'),'rb') as results_file:
    test_results = pickle.load(results_file)
    
    true_classes = test_results.get('true_classes')
    predicted_classes = test_results.get('predicted_classes')
    classes = test_results.get('classes')
    living_classes = test_results.get('living_classes')
    
    accuracy = test_results.get('accuracy')
    balanced_accuracy = test_results.get('balanced_accuracy')
    
    living_precision = test_results.get('living_precision')
    living_recall = test_results.get('living_recall')

### Accuracy scores

In [ ]:
print(f'Accuracy score is {accuracy}')
print(f'Balanced accuracy score is {balanced_accuracy}')
print(f'Living precision score is {living_precision}')
print(f'Living recall score is {living_recall}')

### Confusion matrix

Plot a confusion matrix

In [ ]:
# Create confution matrix
cm = confusion_matrix(true_classes, predicted_classes, normalize='true')

# Plot it
plt.figure(figsize=(20,20))
plt.imshow(cm, cmap='Greys')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=90, fontsize=14)
plt.yticks(tick_marks, classes, fontsize=14)
plt.ylabel('True label', fontsize=14)
plt.xlabel('Predicted label', fontsize=14)
plt.title("Confusion matrix for RF", fontsize=16)
plt.show()

### Classification report

Plot a classification report

In [ ]:
# Create classification report
report = classification_report(true_classes, predicted_classes, output_dict=True)

# List annotations for figure
annot = [str(x) for x in classes]
annot.extend(("accuracy", "macro avg", "weighted avg"))

# Convert report to dataframe
df_report = pd.DataFrame(report).transpose().drop('support', axis=1)

# Plot figure
plt.figure(figsize = (8,15))
sn.heatmap(df_report, annot=True, vmin=0, vmax=1.0,  yticklabels = annot, cmap="Greys")
plt.title("Classification report for RF", fontsize=16)
plt.show()